# Gravitational Wave Analysis and Detection

In this notebook, an implementation is made on the analysis and detection of gravitational waves using spectrogram representation of the samples in the <a href='https://www.kaggle.com/competitions/g2net-gravitational-wave-detection/data'>G2NET dataset</a>.

In this implementation, the spectrogram of the wave signals are computed with the Constant Q Transform algorthim offered by <a href="https://kinwaicheuk.github.io/nnAudio/index.html">nnAudio</a> module.

In [ ]:
!pip install nnAudio

Below the relevant modules for this implementation are imported. A thing to note is that <a href='https://docs.cupy.dev/en/stable/'>cupy</a> and <a href='https://docs.rapids.ai/api/cudf/stable/'>cudf</a> modules are used instead of numpy and pandas, as they are GPU variants and execute much more faster.

In [ ]:
import cupy as cp
import cudf as cd
import os, tqdm, torch, torchvision, gc, random
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from nnAudio.features import CQT

# Data Loading and Exploratory Data Analysis

The dataset consist of '.npy' files aranged in a nested manner inside the train folder. Here the os module is used to iterate efficiently through the nested files to get the file paths and their file names corresponding to the variables *data_paths* and *data_ids* in the program below.

In [ ]:
def get_data_info(root_path: str):
    data_paths = []
    data_ids = []
    for path, subdirs, files in tqdm.tqdm(os.walk(root_path)):
        for name in files:
            data_paths.append(os.path.join(path, name))
            data_ids.append(name.replace('.npy', ''))
            
    return data_paths, data_ids

train_dir = '../input/g2net-gravitational-wave-detection'
data_paths, data_ids = get_data_info(os.path.join(train_dir, 'train'))

Next, the 'training_labels.csv' file is loaded. This spreadsheet file has two columns 'id' and 'target'. The 'id' column corresponds to the file names without their file extensions and the 'target' column contains annotations of 0s and 1s corresponding to whether gravitational wave signal is abscent in the signal waveform or not respectively.

In [ ]:
training_labels = cd.read_csv(os.path.join(train_dir, 'training_labels.csv'))
training_labels.head()

The code below checks for the presence of *NA* values in the 'train_labels.csv' file.

In [ ]:
training_labels.isna().sum()

No *NA* values were found in the labels file. Next a countplot is made to visualise the level of target imbalance in the labels file if any.

In [ ]:
target_0 = training_labels[training_labels['target'] == 0]
target_1 = training_labels[training_labels['target'] == 1]
print(f'length of target 0: {len(target_0)} \nlength of target 1: {len(target_1)}')

ax = sns.countplot(x=training_labels['target'].values.get())

From the plot, it is observed that the target classes are relatively balanced with 280070 0s and 279930 1s.

Below the waveform of some given samples are visualised in a graphical plot. Each sample consists of three waveforms corresponding to simulated signal + noise of three different Interferometers corresponding to Handford LIGO, Livington LIGO and Virgo.

In [ ]:
plot_idx = [0, len(data_paths)//3, len(data_paths)//5, len(data_paths)-1]
sensors = ['handford LIGO', 'livington LIGO', 'virgo']
sample_freq = 2048
timesteps = cp.arange(0, 2, 1/sample_freq).get()

for i, sample_idx in enumerate(plot_idx):
    signals = cp.load(data_paths[sample_idx]).get()
    target = training_labels[training_labels['id'] == data_ids[sample_idx]]['target'].values[0]
    fig = plt.figure(figsize=(20, 4))
    for j, signal in enumerate(signals):
        plt.plot(timesteps, signal, label=f'sensor {sensors[j]}')
        plt.title(f'wave {sample_idx+1}. Target:{target}')
        plt.legend(loc='upper right')
    plt.plot()

The histograms of the selected samples from above are plotted in the next code cell so as to get an overview of the distribution and width of variance of the signal readings of each detector.

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 7))

for i, sample_idx in enumerate(plot_idx):
    axs[i].hist(cp.load(data_paths[sample_idx]).get()[0], label=f'{sensors[0]}')
    axs[i].hist(cp.load(data_paths[sample_idx]).get()[1], label=f'{sensors[1]}')
    axs[i].hist(cp.load(data_paths[sample_idx]).get()[2], label=f'{sensors[2]}')
    axs[i].legend()

It is observed that the distribution of the signal readings of the Virgo detector look skewered with relatively very small variance compared to the Handford and Livington detectors.

In the code cell below, the *SignalDataset()* class is instantiated and is a derived class from the *torch.utils.data.Dataset()* class. This class object is responsible for lazy loading of the dataset. The lazy loading approach was adopted because the dataset was too big and could not fit into the available RAM, making eager loading of large number of samples difficult.

The class returns the spectrogram of any given sample and it's corresponding target class via the *__getitem__()* method. In the *SignalDataset()* class, an object of the *CQT()* class from the *nnAudio* module is instantiated. maximum and minimum frequencies of 20Hz and 500Hz respectively are assumed, the sampling rate (sr) attribute is also set to 2048Hz and the hop length to 64.

The spectrogram is computed in the *preprocess()* method, this method takes the single sample and horizontally stacks the signals of the three differenct detectors into a single vector and computes the spectrogram. The idea behind stacking all signals in a sample horizontally is that a given sample whose target is 1 corresponding to presence of gravitational wave will have similar repeating frequencies along some given time intervals in the spectrogram.

In [ ]:
class SignalDataset(Dataset):
    def __init__(self, paths, ids=None):
        self.paths = paths
        self.ids = ids
        self.CQT_transform = CQT(
            sr=2048,
            fmin=20,           
            fmax=500,
            hop_length=64,
            verbose=False
        )
        
    def __getitem__(self, idx):
        X = cp.load(self.paths[idx])
        X = self.preprocess(X)
        if self.ids != None:
            y = training_labels[training_labels['id'] == self.ids[idx]]['target'].values[0]
            return X, y.get()
        return X
        
    def preprocess(self, signals):
        for i, signal in enumerate(signals):
            signals[i] /= signal.max()
        signals = cp.hstack(signals)
        signal = torch.FloatTensor(signals)
        #shape: C, Freq, Time
        spectrogram = self.CQT_transform(signal)
        return spectrogram
    
    def __len__(self):
        return len(self.paths)

The generated spectrograms of some selected samples are plotted and viewed with the *imshow()* method of the *matplotlib.plyplot()* class below

In [ ]:
training_dataset = SignalDataset(data_paths, data_ids)

for i in plot_idx:
    sample_cqt, target = training_dataset[i]
    fig = plt.figure(figsize=(15, 7))
    plt.imshow(sample_cqt.permute(1, 2, 0))
    plt.title(f'Spectrogram of time domain signal. Target:{target}')
    plt.xlabel('time')
    plt.ylabel('frequency')
    plt.plot()

# Data Modelling with Artifical Neural Networks

In this section the dataset is modelled with the <a href='https://arxiv.org/abs/1512.03385'>ResNet18 </a> neural network architecture with the <a href='https://pytorch.org/docs/stable/index.html'>Pytorch </a> framework. The ResNet18 architecure in this implementation is modified to have a single input channel at first layer, dropout inbetween blocks and a last fc layer with output feature size of 1

In [ ]:
torch.manual_seed(3407)

In [ ]:
class ClassifierNet(torch.jit.ScriptModule):
    
    def __init__(self, input_channels, output_features, dropout):
        super(ClassifierNet, self).__init__()
        self.input_channels = input_channels
        self.output_features = output_features
        self.dropout = dropout
        
        self.resnet = torchvision.models.resnet18(pretrained=False)

        self.resnet.conv1 = nn.Conv2d(
            self.input_channels, 
            64, 
            kernel_size=(7, 7), 
            stride=(2, 2), 
            padding=(3, 3), 
            bias=False)

        self.resnet.fc = nn.Linear(
            in_features=512, 
            out_features=self.output_features, 
            bias=True)
        
        self.add_dropout()
        
        self.sigmoid = nn.Sigmoid()

        
    def add_dropout(self, n_layers=4):
        for i in range(1, n_layers+1):
            layer = list(getattr(self.resnet, f'layer{i}'))
            layer.append(nn.Dropout(self.dropout))
            setattr(self.resnet, f'layer{i}', nn.Sequential(*layer))
        
    
    @torch.jit.script_method
    def forward(self, _input):
        output = self.resnet(_input)
        output = self.sigmoid(output)
        return output


Next the pipeline class is defined for the neural network model training, testing and saving. The *ClassifierPipeline()* class takes as arguments the defined NN model object, the loss function, the optimizer and the device to perform computing on.

In [ ]:
class ClassifierPipeline:
    def __init__(self, classifier, lossfunc, optimizer, device='cpu'):
        
        self.classifier = classifier.to(device)
        self.lossfunc = lossfunc
        self.optimizer = optimizer
        self.device = device
        self.classifier.apply(self.init_weights)
    
    
    def init_weights(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            nn.init.xavier_uniform_(m.weight)
            if m.bias:
                m.bias.data.fill_(0.01)
    
    
    def save_model(self, dirname='./model_params', 
                   filename='gravitational_wave_detection_model.pth.tar'):
        if not os.path.isdir(dirname):os.mkdir(dirname)
        state_dicts = {
            'model_params':self.classifier.state_dict(),
            'optimizer_params':self.optimizer.state_dict(),
        }
        return torch.save(state_dicts, os.path.join(dirname, filename))
        
    
    def train(self, dataloader, epochs=1, verbose=False):
        self.classifier.train()
        losses, accuracies = [], []
        for epoch in range(epochs):
            mean_loss, mean_accuracy = 0, 0
            for idx, (signals, target) in tqdm.tqdm(enumerate(dataloader)):
                signals, target = signals.float().to(self.device), target.float().to(self.device)
                optimizer.zero_grad()
                pred = self.classifier(signals)
                pred = pred.reshape(-1)
                loss = self.lossfunc(pred, target)
                loss.backward()
                self.optimizer.step()
                mean_loss += loss.item()
                target = target.to('cpu').numpy()
                pred = torch.round(pred).detach().to('cpu').numpy()
                accuracy = accuracy_score(target, pred)
                mean_accuracy += accuracy
            mean_loss = mean_loss / (idx+1)
            mean_accuracy = mean_accuracy / (idx+1)
            losses.append(mean_loss)
            accuracies.append(mean_accuracy)
            if verbose:
                print(f'epoch: {epoch} \nLoss: {mean_loss}, \naccuracy: {mean_accuracy}')
        return losses, accuracies
    
    
    def test(self, dataloader, verbose=False, return_cm=False):
        actual_vals, pred_vals = [], []
        self.classifier.eval()
        mean_loss, mean_accuracy = 0, 0
        with torch.no_grad():
            for idx, (signals, target) in tqdm.tqdm(enumerate(dataloader)):
                signals, target = signals.float().to(self.device), target.float().to(self.device)
                pred = self.classifier(signals)
                pred = pred.reshape(-1)
                loss = self.lossfunc(pred, target)
                mean_loss += loss.item()
                target = target.to('cpu').numpy()
                pred = torch.round(pred).detach().to('cpu').numpy()
                accuracy = accuracy_score(target, pred)
                             
                if return_cm:
                    actual_vals.append(target)
                    pred_vals.append(pred)
                    
                mean_accuracy += accuracy
            mean_loss = mean_loss / (idx+1)
            mean_accuracy = mean_accuracy / (idx+1)
            if verbose: print(f'Loss: {mean_loss}, \naccuracy: {mean_accuracy}')
                
        if return_cm: 
            actual_vals = torch.cat([torch.from_numpy(i) for i in actual_vals]).numpy().reshape(-1)
            pred_vals = torch.cat([torch.from_numpy(i) for i in pred_vals]).numpy().reshape(-1)
            cm = confusion_matrix(actual_vals, pred_vals)
            return mean_loss, mean_accuracy, cm
        return mean_loss, mean_accuracy

The *Model_CFG* class is defined to hold configuration values for model, optimizer, training and testing proccess, etc.

In [ ]:
class Model_CFG:
    input_channels = 1
    output_features = 1
    learning_rate = 5e-2
    weight_decay = 5e-7
    dropout = 0.2
    epochs=30
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    batch_size = 400

Next the *data_paths* and *data_ids* list objects are split into training and testing lists which are used to create two objects of the *SignalDataset* class for training and testing. These created *SignalDataset* objects are used to instantiate two *Dataloader* objects for training and testing.

Due to limitations in computing, 40000 samples are used for training and 2000 for testing.

In [ ]:
training_paths, testing_paths, training_ids, testing_ids = train_test_split(
    data_paths, data_ids, shuffle=True, random_state=42, train_size=0.8
)

train_n_samples, test_n_samples = 50000, 5000

training_dataset = SignalDataset(training_paths[:train_n_samples], training_ids[:train_n_samples])
training_dataloader = DataLoader(training_dataset, batch_size=Model_CFG.batch_size, shuffle=True, num_workers=0)

testing_dataset = SignalDataset(testing_paths[:test_n_samples], testing_ids[:test_n_samples])
testing_dataloader = DataLoader(testing_dataset, batch_size=Model_CFG.batch_size, shuffle=True, num_workers=0)

Below, the *ClassifierNet* neural network object is instantiated as well as the loss function and the optimizer.

The loss fuction used in this implentation is the <a href='https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html'>Binary Cross Entropy Loss function</a> and the optimizer used is the <a href='https://pytorch.org/docs/stable/optim.html'>Adam Optimizer</a>

In [ ]:
classifier = ClassifierNet(Model_CFG.input_channels, Model_CFG.output_features, Model_CFG.dropout)     
        
lossfunc = nn.BCELoss()

optimizer = torch.optim.Adam(
    classifier.parameters(), lr=Model_CFG.learning_rate,
    weight_decay=Model_CFG.weight_decay
)

The neural network model architecture is printed below for proper visualisation.

In [ ]:
print(classifier)

The pipeline class is then instantiated and the *classifier*, *lossfunc* and *optimizer* objects are passed. A learning rate scheduler object is also initialised to reduce the learning rate when the measured metric (accuracy in this case) has plateaued after 10 cycles.

Next training and testing commences and the metric values are stored in lists to be visualised later on.

In [ ]:
classifier_pipeline = ClassifierPipeline(
    classifier, lossfunc, optimizer, device=Model_CFG.device
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    classifier_pipeline.optimizer, step_size=10, gamma=0.1, verbose=True)

training_losses, training_accuracies = [], []
testing_losses, testing_accuracies = [], []

best_acc = 0
for i in range(Model_CFG.epochs):
    print(f'epoch: {i}')
    train_loss, train_acc = classifier_pipeline.train(training_dataloader)
    training_losses.append(train_loss[0])
    training_accuracies.append(train_acc[0])
    print(f'training loss: {train_loss[0]}, \ntraining accuracy: {round(train_acc[0]*100, 4)}%')
    
    test_loss, test_acc = classifier_pipeline.test(testing_dataloader)
    testing_losses.append(test_loss)
    testing_accuracies.append(test_acc)
    print(f'testing loss: {test_loss}, \ntesting accuracy: {round(test_acc*100, 4)}% \n')
    
    lr_scheduler.step()
    if test_acc > best_acc:
        best_acc = test_acc
        classifier_pipeline.save_model()
        print(f'model saved at epoch {i}')

The metric values obtained from training and testing are graphically plotted and visualised. These plots are used to get an overview of the overall improvement of the model over the epochs of training.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 6))

axs[0].plot(training_losses, label='training loss')
axs[0].plot(testing_losses, label='testing loss')
axs[0].set_xlabel('epochs')
axs[0].set_ylabel('BCE Loss')
axs[0].set_title('Loss Plot')
axs[0].legend()

axs[1].plot(training_accuracies, label='training accuracy')
axs[1].plot(testing_accuracies, label='testing accuracy')
axs[1].set_xlabel('epochs')
axs[1].set_ylabel('accuracy')
axs[1].set_title('Accuracy Plot')
axs[1].legend()

# Model Evaluation

Finally, the model is evaluated on an evaluation set and the confusion matrix for classification is computed and plotted as shown below.

In [ ]:

eval_n_samples = 1000
eval_dataset = SignalDataset(
    testing_paths[test_n_samples:test_n_samples+eval_n_samples],
    testing_ids[test_n_samples:test_n_samples+eval_n_samples])

eval_dataloader = DataLoader(eval_dataset, batch_size=Model_CFG.batch_size, shuffle=True, num_workers=0)


eval_loss, eval_acc, cm = classifier_pipeline.test(eval_dataloader, return_cm=True)

print(f'evaluation loss: {eval_loss}, \nevaluation accuracy: {round(eval_acc*100, 4)}% \n')

cm_disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
cm_disp.plot()